In [10]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import stanza
from spacy_stanza import StanzaLanguage
from tqdm.notebook import tqdm
import spacy

snlp = stanza.Pipeline(lang="uk")
nlp = StanzaLanguage(snlp)

2020-04-25 20:27:40 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-04-25 20:27:40 INFO: Use device: cpu
2020-04-25 20:27:40 INFO: Loading: tokenize
2020-04-25 20:27:40 INFO: Loading: mwt
2020-04-25 20:27:40 INFO: Loading: pos
2020-04-25 20:27:41 INFO: Loading: lemma
2020-04-25 20:27:41 INFO: Loading: depparse
2020-04-25 20:27:42 INFO: Done loading processors!


In [2]:
def text_prettifier(texts, site):
    prettified = []
    for paragraphs in texts:
        if site == 'kunsht':
            pretty_text = ' '.join([p.replace('\xa0', ' ') for p in paragraphs[:-8] if len(p)>1])
        elif (site == 'nihilist' or site == 'was') and len(paragraphs) > 3:
            pretty_text = ' '.join([p.replace('\xa0', ' ') for p in paragraphs[:-1] if len(p.split(' '))>3])
        elif site == 'lb' and len(paragraphs) > 3:
            pretty_text = ' '.join([p.replace('\xa0', ' ') for p in paragraphs if len(p.split(' '))>3])
        else:
            print ('wring site keyword')
            return
        prettified.append(pretty_text)
    return prettified

def paragraph_prettifier(texts, site):
    prettified_paragraphs = []
    if site == 'kunsht':
        p_slice = slice(-8)
    elif site == 'was' or site == 'nihilist':
        p_slice =slice(-1)
    elif site in ('lb_blogs', 'lb_news'):
        p_slice = slice(None,None)
    for paragraphs in texts:
        for p in paragraphs[p_slice]:
            if len(p) > 3 and len(p.split(' '))>5 and not p.isspace(): 
                    prettified_paragraphs.append(p)
    return prettified_paragraphs

In [3]:
raw_data = {
    'kunsht': joblib.load('kunsht.joblib'), 
    'nihilist': joblib.load('nihilist.joblib'), 
    'lb_blogs': joblib.load('lb_100_pages.joblib'), 
    'was': joblib.load('was_texts.joblib'),
    'lb_news': joblib.load('lb_politycs_news_for_100_pages.joblib'),
}

In [4]:
paragraphs_data = {}

for site, data in raw_data.items():
    print('Number of texts before processing')
    paragraphs_data[site] = paragraph_prettifier(data, site)
    print(site, len(data), len(paragraphs_data[site]))

Number of texts before processing
kunsht 177 4915
Number of texts before processing
nihilist 241 4146
Number of texts before processing
lb_blogs 1786 28809
Number of texts before processing
was 448 10257
Number of texts before processing
lb_news 2195 17492


In [5]:
spacy_docs = {}

In [6]:
#spacy_docs = {}

for site, data in paragraphs_data.items():
    spacy_docs[site] = list(tqdm(nlp.pipe(data[:750])))

In [202]:
def extract_features_with_spacy(doc):
    lemmas = []
    text_without_stops = []
    poses = []
    
    doc_tokens = [w.text for w in doc]
    for word in doc:
        if word not in spacy.lang.uk.stop_words.STOP_WORDS:
            text_without_stops.append(word.text)
        lemmas.append(word.lemma_)
        poses.append(word.pos_)
    text_without_propn = [
        word if pos != 'PROPN' else pos
        for (word, pos)
        in zip(doc_tokens, poses)
    ]
    return text_without_stops, lemmas, poses, text_without_propn

In [203]:
doc = spacy_docs['kunsht'][0]
doc

У землі ховається чимало нових відкриттів. Палеонтолог Вадим Яненко знає це як ніхто. Разом із колегою з Швейцарії Давідом Василяном він описав новий вид саламандр – Palaeoproteus miocenicus. Їхні рештки знайшли зокрема в селі Гриців Хмельницької області. Статтю про відкриття вчених опублікували в журналі Nature. Куншт поговорив з Вадимом про методи дослідження викопних решток і те, як опублікувати своє дослідження в одному з найвідоміших наукових журналів.

In [204]:
t[0].text

'У'

### 2 It's Modeling Time!

In [205]:
site_to_label = {
    'kunsht': ('neutral', 'val'),
    'nihilist': ('subjective', 'val'), 
    'lb_blogs': ('subjective', 'train'), 
    'was': ('neutral', 'val'),
    'lb_news': ('neutral', 'train'),
}

In [206]:
total_texts = []
for site, (label, sample) in site_to_label.items():
    for doc in spacy_docs[site]:
        text_without_stops, lemmas, poses, text_without_propn = extract_features_with_spacy(doc)
        text = doc.text
        total_texts += [
            (site, 
             label, 
             sample, 
             text, 
             ' '.join(text_without_stops), 
             ' '.join(lemmas), 
             ' '.join(poses), 
             ' '.join(text_without_propn)) 
        ]
    print(f'added {site}')

added kunsht
added nihilist
added lb_blogs
added was
added lb_news


In [207]:
df = pd.DataFrame(total_texts, columns=['site', 'label', 'sample', 'text', 'w_stops', 'lemmas', 'poses', 'text_without_propn'])
#val_df = pd.DataFrame(validate, columns=['text', 'label', 'w_stops', 'lemmas', 'poses'])

In [208]:
df.head()

,site,label,sample,text,w_stops,lemmas,poses,text_without_propn
0,kunsht,neutral,val,У землі ховається чимало нових відкриттів. Пал...,У землі ховається чимало нових відкриттів . Па...,у земля ховатися чимало новий відкриття . пале...,ADP NOUN VERB ADV ADJ NOUN PUNCT NOUN PROPN PR...,У землі ховається чимало нових відкриттів . Па...
1,kunsht,neutral,val,"Це була досить цікава історія. Я орнітолог, на...","Це була досить цікава історія . Я орнітолог , ...","це бути досить цікавий історія . я орнітолог ,...",PRON AUX ADV ADJ NOUN PUNCT PRON NOUN PUNCT VE...,"Це була досить цікава історія . Я орнітолог , ..."
2,kunsht,neutral,val,У 2017 році мій колега Олександр Ковальчук поз...,У 2017 році мій колега Олександр Ковальчук поз...,у 2017 рік мій колега Олександр Ковальчук позн...,ADP ADJ NOUN DET NOUN PROPN PROPN VERB PRON AD...,У 2017 році мій колега PROPN PROPN познайомив ...
3,kunsht,neutral,val,"Міоцен – найновіша епоха періоду неогену, друг...","Міоцен – найновіша епоха періоду неогену , дру...","міоцен – найновіший епоха період неогена , дру...",NOUN PUNCT ADJ NOUN NOUN NOUN PUNCT ADJ NOUN A...,"Міоцен – найновіша епоха періоду неогену , дру..."
4,kunsht,neutral,val,"Серед земноводних, що жили на території Україн...","Серед земноводних , що жили на території Украї...","серед земноводний , що жити на територія Украї...",ADP ADJ PUNCT SCONJ VERB ADP NOUN PROPN ADP NO...,"Серед земноводних , що жили на території PROPN..."


In [209]:
df.shape

(3750, 8)

In [210]:
df = df[(~df['text'].str.contains('Рудоманов')) |(~df['text'].str.contains('Ð'))].copy()
#val_df = val_df[~val_df['text'].str.contains('Ð')].copy()

In [211]:
df['label'].value_counts()

neutral       2250
subjective    1500
Name: label, dtype: int64

In [243]:
column_to_try = 'text'

In [244]:
X_train_dev, X_test, y_train_dev, y_test = train_test_split(
    df[df['sample'] == 'train'][column_to_try],
    df[df['sample'] == 'train']['label'],
    random_state=42,
    test_size=0.10,
    stratify=df[df['sample'] == 'train']['label']
)

In [245]:
X_train, X_dev, y_train, y_dev = train_test_split(
    X_train_dev,
    y_train_dev,
    random_state=42,
    test_size=0.20,
    stratify=y_train_dev
)

In [246]:
X_val = df[df['sample']=='val'][column_to_try]
y_val = df[df['sample']=='val']['label']

In [247]:
print(X_train.shape, y_train.shape)

(1080,) (1080,)


In [248]:
y_train.value_counts()

subjective    540
neutral       540
Name: label, dtype: int64

In [249]:
print(classification_report(y_dev, ['subjective']*len(y_dev)))

              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00       135
  subjective       0.50      1.00      0.67       135

    accuracy                           0.50       270
   macro avg       0.25      0.50      0.33       270
weighted avg       0.25      0.50      0.33       270



/home/yevhen/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [250]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, fbeta_score
from sklearn.pipeline import Pipeline

In [251]:
count_lr_clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,3))),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(
        random_state=42, 
        solver = 'sag',
        multi_class='multinomial',
        max_iter=6000,
    )),
])

In [252]:
tf_clf = Pipeline([
    ('vect', TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(
        random_state=42, 
        solver = 'sag',
        multi_class='multinomial',
        max_iter=6000,
    )),
])

In [253]:
tf_rf = Pipeline([
    ('vect', TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', RandomForestClassifier(
        random_state=42, 
        max_depth=10,   
        n_estimators=2000,
        n_jobs=-1
    )),
])

In [254]:
X_train

3120    28 травня 2019 року новий президент Володимир ...
3677    Нагадаємо, 20 січня першим заступником директо...
1851    Навіщо все так ускладнювати? Давайте розберемось.
1530    Проєкт на десяти сторінках. Серед позитиву — у...
3373    Всі ці зміни закладені в законопроєкті про реф...
                              ...                        
3168    Ім'я позивача суд не вказав, але повідомив, що...
1666    Проект передбачає доповнення Регламенту новою ...
2124    Право власності орендар отримує після першого ...
3465    Колишній голова Антитерористичного центру СБУ ...
1773    За проектом закону №3208 Товариства власників ...
Name: text, Length: 1080, dtype: object

In [255]:
count_lr_clf.fit(X_train, y_train);

In [256]:
tf_clf.fit(X_train, y_train);

In [257]:
tf_rf.fit(X_train, y_train);

In [258]:
print('TRAIN PERFORMANCE')
for clf in (count_lr_clf, tf_clf, tf_rf):
    y_pred = clf.predict(X_dev)
    y_pred_proba = clf.predict_proba(X_dev)[:,1]
    f_beta_score = fbeta_score(y_dev, y_pred, beta=0.5, pos_label='subjective')
    print(clf['clf'])
    print(classification_report(y_dev, y_pred))
    print(f'ROC-AUC score = {roc_auc_score(y_dev, y_pred_proba)}')
    print(f'F-0.5 score = {f_beta_score}')
    print('='*80)


TRAIN PERFORMANCE
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=6000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
              precision    recall  f1-score   support

     neutral       0.90      0.95      0.92       135
  subjective       0.94      0.89      0.92       135

    accuracy                           0.92       270
   macro avg       0.92      0.92      0.92       270
weighted avg       0.92      0.92      0.92       270

ROC-AUC score = 0.9636762688614541
F-0.5 score = 0.9331259720062206
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=6000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, s

In [259]:
for clf in (count_lr_clf, tf_clf, tf_rf):
    y_pred = clf.predict(X_val)
    y_pred_proba = clf.predict_proba(X_val)[:,1]
    f_beta_score = fbeta_score(y_val, y_pred, beta=0.5, pos_label='subjective')
    print(clf['clf'])
    print(classification_report(y_val, clf.predict(X_val)))
    print(f'ROC-AUC score = {roc_auc_score(y_val, y_pred_proba)}')
    print(f'F-0.5 score = {f_beta_score}')
    print('='*80)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=6000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
              precision    recall  f1-score   support

     neutral       0.70      0.28      0.40      1500
  subjective       0.35      0.76      0.47       750

    accuracy                           0.44      2250
   macro avg       0.52      0.52      0.44      2250
weighted avg       0.58      0.44      0.43      2250

ROC-AUC score = 0.5710871111111111
F-0.5 score = 0.38761279737489746
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=6000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=

In [265]:
def get_top_n_words(corpus, n=None, ngrams=2):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(
        ngram_range=(ngrams,ngrams),
    ).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]




for ngram in (2,3):
    print(f' NGRAM = {ngram}')
    for label in ('subjective', 'neutral'):
        print(f'LABEL IS {label}')
        label_revs = df[df['label']==label]['text'].values
        print(get_top_n_words(label_revs, 40, ngrams=ngram))
        print(' ')
        
print('='*80)
        
for ngram in (2,3):
    print(f' NGRAM = {ngram}')
    for label in ('subjective', 'neutral'):
        print(f'LABEL IS {label}')
        label_revs = df[df['label']==label]['text_without_propn'].values
        print(get_top_n_words(label_revs, 40, ngrams=ngram))
        print(' ')

 NGRAM = 2
LABEL IS subjective
[('те що', 83), ('тому що', 58), ('під час', 51), ('це не', 51), ('може бути', 42), ('не може', 39), ('2019 року', 34), ('не лише', 32), ('не буде', 31), ('так само', 30), ('про те', 30), ('власників землі', 30), ('того що', 27), ('до того', 26), ('відповідно до', 25), ('млрд грн', 25), ('не тільки', 25), ('не було', 24), ('того щоб', 24), ('не має', 23), ('чи не', 22), ('що не', 22), ('2020 року', 21), ('тому числі', 21), ('тих хто', 21), ('таким чином', 20), ('навіть якщо', 20), ('але не', 20), ('зв язку', 20), ('верховної ради', 20), ('що ми', 19), ('ніколи не', 19), ('covid 19', 19), ('ми не', 18), ('все це', 18), ('закону україни', 18), ('має бути', 18), ('але це', 17), ('того як', 17), ('для того', 17)]
 
LABEL IS neutral
[('під час', 135), ('те що', 85), ('про це', 79), ('тому що', 76), ('про те', 42), ('не було', 36), ('прим ред', 34), ('що це', 32), ('ðµñ ðµð', 32), ('ð½ð ð¾ð', 32), ('крім того', 31), ('до того', 31), ('того як', 31), ('2019 року